In [1]:
import os
import tensorflow as tf
import numpy as np
import random as rn
from deap import base, creator, tools, algorithms
from scipy.stats import randint
import math
import time
import pickle
import random
import operator
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from timeit import default_timer as timer
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
SMALL_SIZE = 15
MEDIUM_SIZE = 18
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE)
plt.rc('axes', titlesize=SMALL_SIZE)
plt.rc('axes', labelsize=MEDIUM_SIZE)
plt.rc('xtick', labelsize=SMALL_SIZE)
plt.rc('ytick', labelsize=SMALL_SIZE)
plt.rc('legend', fontsize=SMALL_SIZE)
plt.rc('figure', titlesize=BIGGER_SIZE)

plt.rcParams['figure.figsize'] = (20.0, 5.0)
sns.set_style("whitegrid")


In [4]:
sd = 7 # Here sd means seed.

def reset_random_seeds():
  os.environ['PYTHONHASHSEED']=str(sd)
  np.random.seed(sd)
  rn.seed(sd)
  tf.random.set_seed(sd)

In [5]:
def save_result(result, filename):
    
    with open(filename, "wb") as output:
        pickle.dump(result, output, pickle.HIGHEST_PROTOCOL)
        
def load_result(filename):
    
    with open(filename, "rb") as input:
        return pickle.load(input)
    
def load_dataset(name='', index=''):
    
    dataframe = pd.read_csv(name)
    dataframe.set_index('date', inplace=True)
    #dataframe=dataframe[dataframe['Year'].between(2015, 2020)]
    
    print('Features:', [i for i in dataframe.columns])
    print('Range: ', dataframe.index[0]," ~ ",dataframe.index[-1])
    return dataframe

def input_output_splitter(series, historical_window:int, prediction_window:int, jump:bool=False, lstm:bool=False):

    input_index = []
    input_value = []
    output_index = []
    output_value = []
    step = 1
    number_predictions = len(series) - historical_window - prediction_window + 1
    
    if jump:
        step = prediction_window
        number_predictions = (len(series) - historical_window)//prediction_window
        
    for i in tqdm(range(number_predictions)):
        input_index.append(series.index[i*step:i*step+historical_window])
        
        if lstm:
            if series.ndim == 1: input_value.append(series.values[i*step:i*step+historical_window].astype('float32'))
            else: input_value.append(series.values[i*step:i*step+historical_window, :].astype('float32'))
        else:
            if series.ndim == 1: input_value.append(series.values[i*step:i*step+historical_window].astype('float32'))
            else: input_value.append(series.values[i*step:i*step+historical_window, :].astype('float32').ravel())

        output_index.append(series.index[i*step+historical_window:i*step+historical_window+prediction_window][0])
        
        
        if series.ndim == 1: output_value.append(series.values[i*step+historical_window:i*step+historical_window+prediction_window].astype('float32'))
        else: output_value.append(series.values[i*step+historical_window:i*step+historical_window+prediction_window, 0].astype('float32'))

    input_index = pd.DataFrame(input_index)
    input_value = np.array(input_value)
    
    if lstm:
        if series.ndim == 1: input_value = input_value[..., np.newaxis]
    
    output_index = pd.DataFrame(output_index, columns=["Date"])
    output_value = np.array(output_value)
    
    return np.array(input_value), input_index, np.array(output_value), output_index

def dataset_splitter(input_data, output_data, data_range, indexes, lstm:bool=False):

    indexes["Date"] = pd.to_datetime(indexes["Date"])
    start = indexes[indexes["Date"] >= data_range[0]].index[0]
    stop = indexes[indexes["Date"] <= data_range[1]].index[-1]
    
    x = input_data[start:stop+1, :, :] if lstm else input_data[start:stop+1, :]
    y = output_data[start:stop+1, :]
        
    return x, y

def dataset_scaling(dataset):
       
    index = dataset.index
    scaler = MinMaxScaler(feature_range=(0, 1))
    
    df = scaler.fit_transform(dataset)
    df = pd.DataFrame(df, columns=dataset.columns, index=index)
    
    return df, scaler

In [6]:
df = load_dataset('procesado.csv')
#df["Weekday"] = (df["Weekday Name"] > 5).astype(int)
pred=24
df["Feriados"] = ((pd.to_datetime(df.index).month== 12) & (pd.to_datetime(df.index).day == 25)).astype(int)
df["Feriados"] = ((pd.to_datetime(df.index).month == 12) & (pd.to_datetime(df.index).day == 24)).astype(int)
df["Feriados"] = ((pd.to_datetime(df.index).month == 12) & (pd.to_datetime(df.index).day == 31)).astype(int)
df["Feriados"] = ((pd.to_datetime(df.index).month == 1) & (pd.to_datetime(df.index).day == 1)).astype(int)
df["SIN_weekpast"] = df["SIN"].shift(periods=(pred*7))
df["temp_t1"] = df["Temperatura Asuncion"].shift(periods=-pred)
df["Year_t1"] = df["Year"].shift(periods=-pred)
df["Hour_t1"] = df["Hour"].shift(periods=-pred)
df["Month_t1"] = df["Month"].shift(periods=-pred)
df["Weekday_t1"] = df["Weekday"].shift(periods=-pred)
df["Feriados_t1"] = df["Feriados"].shift(periods=-pred)
df = df.dropna()

Features: ['Unnamed: 0', 'SIN', 'Temperatura Asuncion', 'Humedad', 'Year', 'Hour', 'Month', 'Quarter', 'Weekday', 'Weekday Name']
Range:  2009-01-01 00:00:00  ~  2021-12-31 21:00:00


In [7]:
dataset=df[[ "SIN","SIN_weekpast","Temperatura Asuncion", "temp_t1", "Year_t1", "Weekday_t1", "Month_t1","Feriados_t1"]]


In [8]:
 dataset.head()

,SIN,SIN_weekpast,Temperatura Asuncion,temp_t1,Year_t1,Weekday_t1,Month_t1,Feriados_t1
date,,,,,,,,
2009-01-08 00:00:00,1307.8,1263.3,26.827256,27.682655,2009.0,4.0,1.0,0.0
2009-01-08 01:00:00,1222.0,1242.3,26.197482,27.546218,2009.0,4.0,1.0,0.0
2009-01-08 02:00:00,1174.0,1210.0,24.502878,26.778438,2009.0,4.0,1.0,0.0
2009-01-08 03:00:00,1129.0,1182.7,23.924053,25.937080,2009.0,4.0,1.0,0.0
2009-01-08 04:00:00,1096.3,1157.5,21.394138,25.408160,2009.0,4.0,1.0,0.0


## LSTM model

In [9]:
def get_hyper_param(n_iteration):
    hyper_param = []  # lookback_size, num_units
    for i in range(n_iteration):
        current_params = []
        current_params.append(np.random.randint(24,168))
        current_params.append(np.random.randint(12,168))

        hyper_param.append(current_params)
    return hyper_param
  


In [10]:
hyper_parameter = get_hyper_param(1000) #En el grid search se tenian 2496 posibilidades
best_params = []
def fit_lstm_grid():
    min_val_loss = 999999
    current_val_loss = 0
    statst= {"lp": [],"ncells": [], "RMSE": [], "MSE": [], "MAPE": [], "10thAPE": [], "R2": [], "MAXAPE": [],"90thAPE": [],"50thAPE": [],"75thAPE": []}
    for eval_look_back,eval_num_units in hyper_parameter:
        #eval_look_back = 117
        #eval_num_units = 162
        dataset_norm, scaler = dataset_scaling(dataset)
        SIN_norm, scalerSin = dataset_scaling(dataset.filter(["SIN"],axis=1))

        X, x_ind, Y, y_ind = input_output_splitter(dataset_norm, historical_window=eval_look_back, 
                                                    prediction_window=pred, jump=True, lstm=True)
        train_range = ('2015-01-01', '2018-12-31')
        val_range =('2019-01-01', '2019-12-31')
        X_train, Y_train = dataset_splitter(X, Y, train_range, y_ind)
        X_val, Y_val = dataset_splitter(X, Y, val_range, y_ind)
        print(f'\nVentana historica: {eval_look_back}\n'
                f'Numero de unidades LSTM: {eval_num_units}')
        n_features = len(dataset_norm.columns)
        
        reset_random_seeds()
        model = Sequential([CuDNNLSTM(eval_num_units, input_shape=(eval_look_back, n_features)),
                            Dense(eval_num_units, activation='relu'),
                            Dense(pred, activation='linear')])

        model.compile(loss="mean_squared_error", optimizer="Adam")

        start = time.time()
        hist = model.fit(X_train, Y_train, epochs=100, shuffle=True, batch_size=64,
                            validation_data=(X_val, Y_val), verbose=0)
        end = time.time()
        print('Time training:', end-start)

        model.summary()
        
        Y_prediction_norm = model.predict(X_val)
        Y_prediction = scalerSin.inverse_transform(Y_prediction_norm)
        Y_valInv = scalerSin.inverse_transform(Y_val)
        rmse=np.sqrt(mean_squared_error(np.array(Y_valInv), Y_prediction)) 
        
        print('RMSE:',rmse)
        print('R2:',r2_score(np.array(Y_valInv), Y_prediction))
        print('MAPE:',mean_absolute_percentage_error(np.array(Y_valInv), Y_prediction))
        MAPE=np.abs((Y_valInv - Y_prediction) /Y_valInv)
        print('10thMAPE:',np.percentile(MAPE,10))
        print('50thMAPE:',np.percentile(MAPE,50))
        print('75thMAPE:',np.percentile(MAPE,75))
        print('90thMAPE:',np.percentile(MAPE,90))
        print('MaxMAPE:',np.max(MAPE))
        
        #  statst= {"lp": [],"ncells": [], "MSE": [], "MAPE": [], "10thAPE": [], "R2": [], "MAXAPE": [],"90thAPE": [],"50thAPE": [],"75thAPE": []}

        statst["lp"].append(eval_look_back)
        statst["ncells"].append(eval_num_units)
        statst["RMSE"].append(rmse)

        statst["MSE"].append(mean_squared_error(np.array(Y_valInv), Y_prediction))
        statst["MAPE"].append(mean_absolute_percentage_error(np.array(Y_valInv), Y_prediction))
        statst["R2"].append(r2_score(np.array(Y_valInv), Y_prediction))
        statst["10thAPE"].append(np.percentile(MAPE,10))
        statst["50thAPE"].append(np.percentile(MAPE,50))
        statst["75thAPE"].append(np.percentile(MAPE,75))
        statst["90thAPE"].append(np.percentile(MAPE,90))
        statst["MAXAPE"].append(np.max(MAPE))

        
        current_val_loss =rmse
        if current_val_loss < min_val_loss:
            min_val_loss = current_val_loss

            best_params = [ eval_look_back, eval_num_units]
            
            #save_result(model,'modelo')

            
            
            print('best_params',best_params)
            
            
        
    return best_params,statst

In [ ]:
start=timer()
best_params,stats = fit_lstm_grid()
end=timer()


100%|██████████| 4738/4738 [00:00<00:00, 10114.05it/s]



Ventana historica: 43
Numero de unidades LSTM: 158
Time training: 27.82558584213257
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm (CuDNNLSTM)       (None, 158)               106176    
_________________________________________________________________
dense (Dense)                (None, 158)               25122     
_________________________________________________________________
dense_1 (Dense)              (None, 24)                3816      
Total params: 135,114
Trainable params: 135,114
Non-trainable params: 0
_________________________________________________________________
RMSE: 162.06168
R2: 0.8463228513537576
MAPE: 0.06099586
10thMAPE: 0.008470607455819845
50thMAPE: 0.045803628861904144
75thMAPE: 0.08229045011103153
90thMAPE: 0.1291547268629074
MaxMAPE: 0.50061285
best_params [43, 158]


100%|██████████| 4737/4737 [00:00<00:00, 10173.16it/s]



Ventana historica: 60
Numero de unidades LSTM: 149
Time training: 29.154820680618286
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_1 (CuDNNLSTM)     (None, 149)               94764     
_________________________________________________________________
dense_2 (Dense)              (None, 149)               22350     
_________________________________________________________________
dense_3 (Dense)              (None, 24)                3600      
Total params: 120,714
Trainable params: 120,714
Non-trainable params: 0
_________________________________________________________________
RMSE: 170.14944
R2: 0.8258544260274642
MAPE: 0.062051475
10thMAPE: 0.0092215146869421
50thMAPE: 0.05055498145520687
75thMAPE: 0.08594221249222755
90thMAPE: 0.122923593968153
MaxMAPE: 0.5314155


100%|██████████| 4736/4736 [00:00<00:00, 11690.39it/s]


...


Ventana historica: 71
Numero de unidades LSTM: 41
Time training: 26.108684539794922
Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_51 (CuDNNLSTM)    (None, 41)                8364      
_________________________________________________________________
dense_102 (Dense)            (None, 41)                1722      
_________________________________________________________________
dense_103 (Dense)            (None, 24)                1008      
Total params: 11,094
Trainable params: 11,094
Non-trainable params: 0
_________________________________________________________________
RMSE: 218.47617
R2: 0.7328764463665772
MAPE: 0.09684634
10thMAPE: 0.01509113796055317
50thMAPE: 0.0794297307729721
75thMAPE: 0.1361878179013729
90thMAPE: 0.20477116107940674
MaxMAPE: 0.67997336


100%|██████████| 4738/4738 [00:00<00:00, 10341.36it/s]



Ventana historica: 36
Numero de unidades LSTM: 121
Time training: 23.9954252243042
Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_52 (CuDNNLSTM)    (None, 121)               63404     
_________________________________________________________________
dense_104 (Dense)            (None, 121)               14762     
_________________________________________________________________
dense_105 (Dense)            (None, 24)                2928      
Total params: 81,094
Trainable params: 81,094
Non-trainable params: 0
_________________________________________________________________
RMSE: 169.73216
R2: 0.8269818775723147
MAPE: 0.062338393
10thMAPE: 0.009362776298075914
50thMAPE: 0.049126045778393745
75thMAPE: 0.0849030613899231
90thMAPE: 0.12697602063417435
MaxMAPE: 0.56037015


100%|██████████| 4736/4736 [00:00<00:00, 10168.55it/s]



Ventana historica: 88
Numero de unidades LSTM: 127


In [12]:
print("Time Taken -> ",str((end-start)/60/60))

print("Best params ",best_params)

Time Taken ->  9.771513602594172
Best params  [164, 151]


In [ ]:
from keras.models import load_model
model = load_model('modelo.h5')
model.summary()

In [ ]:
X_train = pd.read_pickle('x_train.pkl')
Y_train = pd.read_pickle('y_train.pkl')
X_val = pd.read_pickle('x_val.pkl')
Y_val = pd.read_pickle('y_valscaler.pkl')

Y_prediction_norm = pd.read_pickle('y_prediction_norm.pkl')
Y_prediction = pd.read_pickle('y_prediction.pkl')
Y_valInv = pd.read_pickle('y_valInv.pkl')


SIN_norm, scalerSin = dataset_scaling(dataset.filter(["SIN"],axis=1))

Y_prediction_norm = model.predict(X_val)
Y_prediction = scalerSin.inverse_transform(Y_prediction_norm)
Y_valInv = scalerSin.inverse_transform(Y_val)
rmse=np.sqrt(mean_squared_error(np.array(Y_valInv), Y_prediction)) 
        
model.summary()
    
print('RMSE:',rmse)
print('R2:',r2_score(np.array(Y_valInv), Y_prediction))
print('MAPE:',mean_absolute_percentage_error(np.array(Y_valInv), Y_prediction))
MAPE=np.abs((Y_valInv - Y_prediction) /Y_valInv)
print('10thMAPE:',np.percentile(MAPE,10))
print('50thMAPE:',np.percentile(MAPE,50))
print('75thMAPE:',np.percentile(MAPE,75))
print('90thMAPE:',np.percentile(MAPE,90))
print('MaxMAPE:',np.max(MAPE))

In [13]:
df=pd.DataFrame.from_dict(stats)
df.to_csv('RandomSearch9.7horas.csv', index=False)